# Generate Data for Chartbook

Brian Dew

@bd_econ

In [1]:
import sys
sys.path.append('../src')

import time

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### S&P 500 

In [2]:
ltdate = int(time.time())

url = ('https://query1.finance.yahoo.com/v7/finance/download/%5EGSPC?'+
       f'period1=599616000&period2={ltdate}&interval=1d&events=history')

df = pd.read_csv(url).set_index('Date')

close = df['Adj Close']
close.index = pd.to_datetime(close.index)

final = close.resample('MS').mean().append(pd.Series({close.index[-1]: close[-1]}))
final.name = 'GSPC'
final.to_csv(data_dir / 'sp500.csv', index_label='date', float_format='%g') 

data = pd.DataFrame()
data['GSPC'] = close.loc['2017':]
data['MA'] = close.rolling(253).mean().loc['2017':]
data.to_csv(data_dir / 'sp500_recent.csv', index_label='date', float_format='%g')

ltdate = dtxt(close.index[-1])['day1']
ltval = close.iloc[-1]
hival = df.High.iloc[-253:].max()
chhi = (1 - ltval/hival) * 100
hidt = dtxt(df.High.iloc[-253:].idxmax())['day1']
loval = df.Low.iloc[-253:].min()
chlo = abs((1 - ltval/loval) * 100)
lodt = dtxt(df.Low.iloc[-253:].idxmin())['day1']
avval = df['Adj Close'].iloc[-253:].mean()
chav = (1 - ltval/avval) * 100
chavabs = abs(chav)
chtype = 'below' if chav > 0 else 'above'

text = (f'The S\&P 500 closed at {ltval:.0f} on {ltdate}. The index is currently '+
        f'{chhi:.1f} percent below its one-year high of {hival:.0f} on {hidt}, and '+
        f'{chlo:.1f} percent above its one-year low of {loval:.0f} on {lodt}. The '+
        f'average over the past year is {avval:.0f}; the index is {chavabs:.1f} percent '+
        f'{chtype} its one-year moving average '+
         '(see {\color{black!18!white}\\textbf{---}}).')
write_txt(text_dir / 'sp500.txt', text)
print(text)

The S\&P 500 closed at 3692 on December 7, 2020. The index is currently 0.2 percent below its one-year high of 3699 on December 4, 2020, and 68.4 percent above its one-year low of 2192 on March 23, 2020. The average over the past year is 3184; the index is 16.0 percent above its one-year moving average (see {\color{black!18!white}\textbf{---}}).


### Real Interest Rates

In [3]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H15&series=df361c47287a0589560a46dad7d610cb&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)

n = {'RIFLGFCY10_XII_N.B': 'Ten-year',
     'RIFLGFCY05_XII_N.B': 'Five-year'}

df = clean_data[n.keys()].rename(n, axis=1).dropna(subset=['Ten-year'])

data = (df.resample('MS').mean().iloc[:-1].append(df.iloc[-1]))
data.to_csv(data_dir / 'real_rates.csv', index_label='date', float_format='%g')

In [4]:
ltdate = dtxt(df.index[-1])['day1']
prdate = dtxt(df.index[-22])['day1']
ltval = df['Ten-year'].iloc[-1]
ltval5 = df['Five-year'].iloc[-1]
prval = df['Ten-year'].iloc[-22]
prval5 = df['Five-year'].iloc[-22]

text = ('US Treasury inflation-indexed securities are used to measure real '+
        'interest rates. Treasury yields in general are a good proxy for '+
        'low-risk rates, and the treasury offers specific securities that '+
        'have interest payments indexed to the consumer price index (CPI) rate '+
        'of inflation. \n\n The real yield for such treasuries with ten years '+
        f'to maturity is {ltval:.2f} percent, as of {ltdate} '+
        '(see {\color{red}\\textbf{---}}), '+
        f'compared to {prval:.2f} '+
        f'percent on {prdate} . For a five-year maturity, the real yield is '+
        f'{ltval5:.2f} percent in the latest data '+
        '(see {\color{violet}\\textbf{---}}), compared to '+
        f'{prval5:.2f} percent a month prior. ')
write_txt(text_dir / 'real_rates_basic.txt', text)
print(text)

US Treasury inflation-indexed securities are used to measure real interest rates. Treasury yields in general are a good proxy for low-risk rates, and the treasury offers specific securities that have interest payments indexed to the consumer price index (CPI) rate of inflation. 

 The real yield for such treasuries with ten years to maturity is -0.92 percent, as of December 4, 2020 (see {\color{red}\textbf{---}}), compared to -0.82 percent on November 3, 2020 . For a five-year maturity, the real yield is -1.41 percent in the latest data (see {\color{violet}\textbf{---}}), compared to -1.23 percent a month prior. 


### Interest rates

In [5]:
# Taylor Rule suggested Fed Funds rate
p = (pd.read_csv(data_dir / 'pce_pi.csv', parse_dates=['date'])
       .set_index('date')['CORE']).resample('QS').mean() * 0.5

y = ((nipa_df(retrieve_table('T10106')['Data'], ['A191RX']))
      .loc['1989':, 'A191RX'])

y_p = fred_df('GDPPOT')['VALUE'] * 1_000

o = (y - (y_p)).divide(y_p).dropna()

taylor_ff = (p + 3 + (0.5*(p - 2)) + (1*(o*100)))

taylor_ff.name = 'Value'

taylor_ff.dropna().to_csv(data_dir / 'taylor.csv', index_label='date')

In [6]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H15&series=a2738d013c65f34c46f905d1d7913f78&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)

n = {'RIFLGFCY10_N.B': 'Ten-year',
    'RIFLGFCY30_N.B': 'Thirty-year',
    'RIFSPFF_N.B': 'Fed Funds',
    'RIFLGFCM03_N.B': 'Three-month',
    'RIFLGFCY05_N.B': 'Five-year',
    'RIFLGFCY02_N.B': 'Two-year',
    'RIFLGFCM01_N.B': 'One-month',
    'RIFLGFCY01_N.B': 'One-year',
    'RIFLGFCY20_N.B': 'Twenty-year'}

df = clean_data[n.keys()].rename(n, axis=1).dropna(subset=['Ten-year'])

data = (df.resample('MS').mean().iloc[:-1].append(df.iloc[-1]))
data.to_csv(data_dir / 'rates.csv', index_label='date', float_format='%g')

ldate = dtxt(data.index[-1])['day1']
tenlt = df['Ten-year'].iloc[-1]
tenpr = df['Ten-year'].iloc[-252]
twolt = df['Two-year'].iloc[-1]
twopr = df['Two-year'].iloc[-252]

text = (f'As of {ldate}, the constant maturity yield for a ten-year '+
        'US Treasury bond (see {\color{red!90}\\textbf{---}}) '+
        f'is {tenlt} percent, compared to {tenpr:.2f} percent one year prior. '+
        'The yield for a two-year Treasury (see {\color{blue}\\textbf{---}}) '
        +f'is {twolt} percent, compared to {twopr:.2f} '+
        f'percent a year prior.')
write_txt(text_dir / 'rates_basic.txt', text)
print(text)

text = ('The effective fed funds rate (see {\color{blue!60!black}\\textbf{---}}) '+
        f'is {data["Fed Funds"].iloc[-1]} percent, as of {ldate}.')
write_txt(text_dir / 'rates_ff.txt', text)
print(text)

rows = ['Three-month', 'Two-year', 'Five-year', 'Ten-year', 'Thirty-year']
columns = [-1, -2, -5, -22, -64, -127, -252, -1251]
data2 = df[rows].iloc[columns].T
data2.columns = [dtxt(i)['day2'] for i in pd.to_datetime(data2.keys())]

(data2.applymap('{:,.2f}'.format)
      .to_csv(data_dir / 'treasury_rates.tex', sep='&', 
           line_terminator='\\\ ', quotechar=' '))

data2

As of December 4, 2020, the constant maturity yield for a ten-year US Treasury bond (see {\color{red!90}\textbf{---}}) is 0.97 percent, compared to 1.77 percent one year prior. The yield for a two-year Treasury (see {\color{blue}\textbf{---}}) is 0.16 percent, compared to 1.58 percent a year prior.
The effective fed funds rate (see {\color{blue!60!black}\textbf{---}}) is 0.09 percent, as of December 4, 2020.


,"Dec 4, 2020","Dec 3, 2020","Nov 30, 2020","Nov 3, 2020","Sep 2, 2020","Jun 4, 2020","Dec 4, 2019","Dec 4, 2015"
Three-month,0.09,0.08,0.08,0.10,0.12,0.15,1.55,0.23
Two-year,0.16,0.16,0.16,0.17,0.14,0.19,1.58,0.96
Five-year,0.42,0.40,0.36,0.39,0.26,0.40,1.60,1.71
Ten-year,0.97,0.92,0.84,0.90,0.66,0.82,1.77,2.28
Thirty-year,1.73,1.67,1.58,1.66,1.38,1.61,2.22,3.01


### Yield Curve

In [7]:
i = {'One-month': 1, 'Three-month': 2, 'One-year': 3, 'Two-year': 4, 
     'Five-year': 5, 'Ten-year': 6, 'Twenty-year': 7, 'Thirty-year': 8}
tbl = pd.DataFrame()
for v, c in [(-1, 'value'), (-252, 'oneyear'), (-252*5, 'fiveyear')]:
    col = df[i.keys()].iloc[v]
    col.index = col.index.map(i)
    tbl[c] = col
tbl.index.name = 'number'
tbl['alignment'] = 270

tbl.to_csv(data_dir / 'yc.csv', float_format='%g')

### Spread

In [8]:
data = df[df['Ten-year'] != 'ND'].astype('float')
spread = pd.DataFrame()
spread['Ten-3M'] = data['Ten-year'] - data['Three-month']
spread['Ten-2Y'] = data['Ten-year'] - data['Two-year']
spread.loc['2012':].to_csv(data_dir / 'spread.csv', index_label='date', float_format='%g', header=True)

node = end_node(spread['Ten-3M'], 'cyan!60!blue').replace('\\%', '')
write_txt(text_dir / 'spread_node.txt', node)

node = end_node(spread['Ten-2Y'], 'orange', ).replace('\\%', '')
write_txt(text_dir / 'spread_node2.txt', node)

lt3 = spread['Ten-3M'].iloc[-1]
lt3t = f'{lt3:.2f} {["percentage points" if lt3 > 1.05 else "percentage point"][0]}'
pr3 = spread['Ten-3M'].iloc[-252]
pr3t = f'{pr3:.2f} {["percentage points" if pr3 > 1.05 else "percentage point"][0]}'
lt2 = spread['Ten-2Y'].iloc[-1]
lt2t = f'{lt2:.2f} {["percentage points" if lt2 > 1.05 else "percentage point"][0]}'
pr2 = spread['Ten-2Y'].iloc[-252]
pr2t = f'{pr2:.2f} {["percentage points" if pr2 > 1.05 else "percentage point"][0]}'

text = (f'As of {ldate}, the spread between a 10-year treasury bond and '+
        f'a three-month treasury bill is {lt3t} '+
        '(see {\color{cyan!60!blue}\\textbf{---}}), compared to '+
        f'{pr3t} one year prior. The spread between 10-year '+
        'and 2-year treasuries (see {\color{orange}\\textbf{---}}) '+
        f'was {lt2t} on {ldate}, and '+
        f'{pr2t} one year prior.')

write_txt(text_dir / 'spread_basic.txt', text)
print(text)

As of December 4, 2020, the spread between a 10-year treasury bond and a three-month treasury bill is 0.88 percentage point (see {\color{cyan!60!blue}\textbf{---}}), compared to 0.22 percentage point one year prior. The spread between 10-year and 2-year treasuries (see {\color{orange}\textbf{---}}) was 0.81 percentage point on December 4, 2020, and 0.19 percentage point one year prior.


### Gold Price

In [9]:
df = fred_df('GOLDAMGBD228NLBM').dropna()
data = (df.resample('M').mean().iloc[:-1].append(df.iloc[-1]))
data.to_csv(data_dir / 'gold.csv', index_label='date')

ltdate = dtxt(data.index[-1])['day1']
ltval = data.VALUE.iloc[-1]
prdate = dtxt(data.index[-13])['mon1']
prval = data.VALUE.iloc[-13]
grdate = dtxt(data.loc['2006':'2011', 'VALUE'].idxmax())['mon1']
grval = data.loc['2006':'2011', 'VALUE'].max()

text = (f'As of {ltdate}, one troy ounce of gold sells for \${ltval:,.2f} '+
        '(see {\color{orange!40!yellow}\\textbf{---}}), '
        f'compared to an average of \${prval:,.2f} per ounce during {prdate}. '+
        'Following the great recession, the monthly average price '+
        f'of gold reached \${grval:,.2f} per ounce, in {grdate}.')
print(text)
write_txt(text_dir / 'gold.txt', text)

As of December 4, 2020, one troy ounce of gold sells for \$1,839.30 (see {\color{orange!40!yellow}\textbf{---}}), compared to an average of \$1,480.03 per ounce during December 2019. Following the great recession, the monthly average price of gold reached \$1,780.65 per ounce, in September 2011.


### Fed Balance Sheet

In [10]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H41&series=38d757c01cb3b550176f371352643679&lastobs=&'
dt = 'from=01/01/2002&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

raw_data = pd.read_csv(url)

d = {v: re.sub("\s+[\(\[].*?[\)\]]", "", i.split(';')[0]) 
     for i, v in raw_data.iloc[4, 1:].iteritems()}

date_column = raw_data.loc[5:, 'Series Description']
date_index = pd.to_datetime(date_column).rename('Date')
columns = raw_data.iloc[4, 1:].values
    
clean_data = raw_data.iloc[5:, 1:].replace('ND', np.nan).astype('float')
clean_data.index = date_index
clean_data.columns = columns

clean_data = clean_data / 1_000_000

clean_data.to_csv(data_dir / 'fed_assets.csv', index_label='date', float_format='%g')

In [11]:
ltval = clean_data['RESPPMA_N.WW'].iloc[-1].round(1)
ltdate = dtxt(clean_data.index[-1])['day1']
n = (clean_data.iloc[-1] - clean_data.iloc[-2])['RESPPMA_N.WW'] * 1000
chval = (f'increased by \${abs(n):.0f} billion' if n >= 10 
         else f'decreased by \${abs(n):.0f} billion' 
         if n <= -10 else 'was largely unchanged')

text = ('In response to the collapse of the housing bubble, the Fed purchased '+
        'U.S. Treasury bonds and mortgage-backed securities. Total assets held by '+
        'the Federal Reserve (see {\color{blue!80!black}\\textbf{---}}) '+
        'increased from \$0.9 trillion in August 2008 to '+
        '\$2.2 trillion in November 2008. Additional rounds of asset purchases, '+
        'referred to as quantitative easing, increased the balance sheet to '+
        '\$4.5 trillion by January 2014. As bonds mature they were replaced '+
        'until October 2017, when the Fed allowed the size of its balance '+
        'sheet to normalize. Total assets fell below \$3.8 trillion in August 2019.')

write_txt(text_dir / 'fed_assets1.txt', text)
        
        
txt2 = ('Balance sheet normalization ended in September 2019 when the Fed increased '+
        'operations in overnight and term repurchase agreement (repo) markets, following '+
        'a sharp increase in rates in these markets. The Fed balance sheet '+
        'increased to \$4.1 trillion by December 2019. More recently, in response to '+
        'worsening financial conditions, the Fed began to purchase commercial bonds '+
        'and to offer currency swaps with major US trading partners. '+
        'The Fed balance sheet increased from \$4.2 trillion in February 2020 to '+
        f'\${ltval} trillion, as of the latest data, covering {ltdate}. The total value of '+
        f'Fed assets {chval} from the value one week prior.')

write_txt(text_dir / 'fed_assets2.txt', txt2)

print(txt2)

Balance sheet normalization ended in September 2019 when the Fed increased operations in overnight and term repurchase agreement (repo) markets, following a sharp increase in rates in these markets. The Fed balance sheet increased to \$4.1 trillion by December 2019. More recently, in response to worsening financial conditions, the Fed began to purchase commercial bonds and to offer currency swaps with major US trading partners. The Fed balance sheet increased from \$4.2 trillion in February 2020 to \$7.2 trillion, as of the latest data, covering December 2, 2020. The total value of Fed assets was largely unchanged from the value one week prior.


### Fed Balance Sheet Table

In [12]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H41&series=bbb7b50f663fe933c8cce7e3707e3998&lastobs=&'
dt = 'from=01/01/2002&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)

d2 = {'RESPPA_N.WW': '\\textbf{Total} (see {\color{blue!80!black}\\textbf{---}})',
      'RESPPALGUO_N.WW': '\hspace{2mm}U.S. Treasury securities',
      'RESPPALGASMO_N.WW': '\hspace{2mm}Mortgage-backed securities',
      'RESH4SCS_N.WW': '\hspace{2mm}Central bank liquidity swaps',
      'RESPPALGTR_N.WW': '\hspace{2mm}Repurchase agreements',
      'RESPPALD_N.WW': '\hspace{2mm}Loans',
      'RESPPALDJ_N.WW': '\hspace{4mm}Payroll Protection Program',
      'NetPremDisc': '\hspace{2mm}Net unamortized premium',
      'Other': '\hspace{2mm}Other'}

pr_di = ['RESPPALSD_N.WW', 'RESPPALSP_N.WW']

data = clean_data.copy()
data['Other'] = (data['RESPPA_N.WW'] - 
                             data.drop('RESPPA_N.WW', axis=1).sum(axis=1))
data['NetPremDisc'] = data[pr_di].sum(axis=1)

data = data.rename(d2, axis=1).drop(pr_di, axis=1)

df = pd.DataFrame()

for i in [-1, -2, -5, -14, -53]:
    df[dtxt(data.index[i])['day2']] = data.iloc[i]
    
(df.loc[list(d2.values()), :].div(1000)
   .applymap('{:,.1f}'.format)
   .to_csv(data_dir / 'fed_bal_sheet.tex', sep='&', 
           line_terminator='\\\ ', quotechar=' '))

### Money Growth

In [13]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H6&series=fafc1295c552e99d2b907eb62278e4ca&lastobs=&'
dt = 'from=01/01/1988&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

raw_data = pd.read_csv(url)

d = {v: re.sub("\s+[\(\[].*?[\)\]]", "", i) 
     for i, v in raw_data.iloc[4, 1:].iteritems()}

date_column = raw_data.loc[5:, 'Series Description']
date_index = pd.to_datetime(date_column).rename('Date')
columns = raw_data.iloc[4, 1:].values
    
clean_data = raw_data.iloc[5:, 1:].replace('ND', np.nan).astype('float')
clean_data.index = date_index
clean_data.columns = columns

final2 = {}
data = clean_data[['M2_N.WM', 'MMFIN_N.WM']].sum(axis=1)
month_list = clean_data.resample('MS').mean().pct_change(12).dropna().index
short_month = ''
for date in month_list:
    month_len = len(data.loc[date.strftime('%Y-%m')])
    prevyr = f'{date.year - 1}-{date.month}'
    weeks_in_short_month = 4
    end_date = date
    if month_len < 4:
        end_date = date
        short_month = date.strftime('%B %Y')
        val = data.loc[date.strftime('%Y-%m')].mean()
        prv = data.loc[prevyr].iloc[:month_len]
        prev = prv.mean()
        weeks_in_short_month = len(prv)
    elif month_len >= 4:
        val = data.loc[date.strftime('%Y-%m')].mean()
        prev = data.loc[prevyr].mean()
        end_full = date
    final2[date] = (val / prev - 1) * 100
    
final = pd.Series(final2, name='value')
final.to_csv(data_dir / 'm2imf.csv', index_label='date', header='True')

s = series_info(final)

node = end_node(final, 'green!80!blue')
write_txt(text_dir / 'm2imf_node.txt', node)

week_conv = {1: f'the first week of {short_month}', 
             2: f'the first two weeks of {short_month}', 
             3: f'the first three weeks of {short_month}',
             4: end_full.strftime('%B %Y')}

if final.iloc[-1] >= 0.1:
    txt = f'increased over the equivalent previous year value by {final.iloc[-1]:.1f} percent'
elif final.iloc[-1] <= -0.1:
    txt = f'decreased over the equivalent previous year value by {abs(final.iloc[-1]):.1f} percent'   
else:
    txt = 'was virtually unchanged over the previous year value'
    
if s['days_since_match'] > 300:
    txt2 = f", {s['last_matched'].replace('highest level', 'fastest growth rate')}."
else:
    txt2 = '.'
    
text = (f'In {week_conv[weeks_in_short_month]}, '+
        f'the M2 plus institutional money funds measure {txt}{txt2}')

write_txt(text_dir / 'm2imf.txt', text)

gdp = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC']
m2sh = (clean_data['M2_N.WM'].iloc[-1] / (gdp.iloc[-1] / 1000)) * 100
imfsh = (clean_data['MMFIN_N.WM'].iloc[-1] / (gdp.iloc[-1] / 1000)) * 100

text2 = (f'In the week of {clean_data.index[-1].strftime("%B %-d, %Y")}, '+
         f'the M2 measure of money averaged \${clean_data["M2_N.WM"].iloc[-1] / 1000:.1f} '+
         f'trillion, equivalent to {m2sh:.1f} percent of GDP. Institution money market '
         +f'accounts, which are not included in M2, can be combined with M2 to create a '+
         f'slightly-broader-than-M2 measure of the money stock. These funds averaged '+
         f'\${clean_data["MMFIN_N.WM"].iloc[-1] / 1000:.1f} trillion in the same week, '+
         f'equivalent to {imfsh:.1f} percent of GDP. ')

write_txt(text_dir / 'm2imf2.txt', text2)
print(text, '\n')
print(text2)

In November 2020, the M2 plus institutional money funds measure increased over the equivalent previous year value by 25.2 percent. 

In the week of November 23, 2020, the M2 measure of money averaged \$19.1 trillion, equivalent to 90.3 percent of GDP. Institution money market accounts, which are not included in M2, can be combined with M2 to create a slightly-broader-than-M2 measure of the money stock. These funds averaged \$2.9 trillion in the same week, equivalent to 13.5 percent of GDP. 


### Exchange rates

In [10]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H10&series=ad1712193ad5bad7b424e3ae5eb101a5&lastobs=&'
dt = 'from=01/01/1989&to=12/31/2020&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

raw_data = pd.read_csv(url)

d = {v: re.sub("\s+[\(\[].*?[\)\]]", "", i.split(';')[0]) 
     for i, v in raw_data.iloc[4, 1:].iteritems()}

date_column = raw_data.loc[5:, 'Series Description']
date_index = pd.to_datetime(date_column).rename('Date')
columns = raw_data.iloc[4, 1:].values
    
clean_data = raw_data.iloc[5:, 1:].replace('ND', np.nan).astype('float')
clean_data.index = date_index
clean_data.columns = columns

for cc in ['EU', 'UK', 'AL', 'NZ']:
    clean_data[f'RXI_N.B.{cc}'] = 1 / clean_data[f'RXI$US_N.B.{cc}'] 
clean_data['RXI_N.B.JA'] = clean_data['RXI_N.B.JA'] / 100.0

latest = clean_data.dropna(how='all').iloc[-1]
major = ['RXI_N.B.EU', 'RXI_N.B.UK', 'RXI_N.B.CA', 'RXI_N.B.JA']
twidx = clean_data.resample('MS').mean().append(latest)['JRXWTFB_N.B'].dropna()
clean_data.resample('MS').mean().append(latest)[major].to_csv(data_dir / 'fx1.csv', index_label='date', float_format='%g')
twidx.to_csv(data_dir / 'fx_idx.csv', index_label='date', float_format='%g', header=True)

ldate = dtxt(twidx.index[-1])['day1']
lval = twidx.iloc[-1]
totch = ((lval / 100) - 1) * 100
threeyr = twidx.iloc[-38:].mean()
prev3yr = twidx.iloc[-74:-38].mean()

text = (f'The latest index value, as of {ldate}, is {lval:.1f}, an increase '+
        f'of {totch:.1f} percent since inception in 2006. Over the past three years, '+
        f'the index value has averaged {threeyr:.1f}, compared to an average '+
        f'of {prev3yr:.1f} over the previous three-year period.')
write_txt(text_dir / 'twdbasic.txt', text)
print(text)

The latest index value, as of December 4, 2020, is 112.1, an increase of 12.1 percent since inception in 2006. Over the past three years, the index value has averaged 115.0, compared to an average of 111.1 over the previous three-year period.


In [11]:
cad = clean_data.loc[ldate, 'RXI_N.B.CA']
jpy = clean_data.loc[ldate, 'RXI_N.B.JA'] * 100
eur = clean_data.loc[ldate, 'RXI_N.B.EU']
gbp = clean_data.loc[ldate, 'RXI_N.B.UK']
cadpc = inc_dec_percent(clean_data['RXI_N.B.CA'].pct_change(262).iloc[-1] * 100)
jpypc = inc_dec_percent(clean_data['RXI_N.B.JA'].pct_change(262).iloc[-1] * 100)
gbppc = inc_dec_percent(clean_data['RXI_N.B.UK'].pct_change(262).iloc[-1] * 100)
eurpc = inc_dec_percent(clean_data['RXI_N.B.EU'].pct_change(262).iloc[-1] * 100)


text = (f'As of {ldate}, one US dollar buys approximately: {cad:.2f} Canadian dollars '+
        f'(see {{\color{{green!85!blue}}\\textbf{{---}}}}), {jpy:.0f} Japanese Yen '+
        f'(see {{\color{{red}}\\textbf{{---}}}}), {eur:.2f} Euros '+
        f'(see {{\color{{cyan!90!white}}\\textbf{{---}}}}), and {gbp:.2f} British Pounds '+
         '(see {\color{blue!90!cyan}\\textbf{---}}). Over the past three years, the nominal '+
        f'exchange rate between the US dollar and the Canadian dollar {cadpc}, the USD-JPY rate '+
        f'{jpypc}, the USD-EUR rate {eurpc}, and the USD-GBP rate {gbppc}.')
write_txt(text_dir / 'selcurr_basic.txt', text)
print(text)

As of December 4, 2020, one US dollar buys approximately: 1.28 Canadian dollars (see {\color{green!85!blue}\textbf{---}}), 104 Japanese Yen (see {\color{red}\textbf{---}}), 0.82 Euros (see {\color{cyan!90!white}\textbf{---}}), and 0.74 British Pounds (see {\color{blue!90!cyan}\textbf{---}}). Over the past three years, the nominal exchange rate between the US dollar and the Canadian dollar decreased by 3.1 percent, the USD-JPY rate decreased by 4.3 percent, the USD-EUR rate decreased by 8.8 percent, and the USD-GBP rate decreased by 2.9 percent.


### Exchange Rates Table 

In [12]:
fx = {'RXI_N.B.EU': 'EUR',
      'RXI_N.B.UK': 'GBP',
      'RXI_N.B.JA': 'JPY',
      'RXI_N.B.CA': 'CAD',
      'RXI_N.B.MX': 'MXN',
      'RXI_N.B.CH': 'CNY',
      'RXI_N.B.SZ': 'CHF',
      'RXI_N.B.HK': 'HKD',
      'RXI_N.B.IN': 'INR',
      'RXI_N.B.AL': 'AUD',
      'RXI_N.B.NZ': 'NZD',
      'RXI_N.B.BZ': 'BRL',
      'RXI_N.B.KO': 'KRW',
      'RXI_N.B.MA': 'MYR',
      'RXI_N.B.DN': 'DKK',
      'RXI_N.B.NO': 'NOK',
      'RXI_N.B.SD': 'SEK',
      'RXI_N.B.SF': 'ZAR',
      'RXI_N.B.SI': 'SGD',
      'RXI_N.B.TA': 'TWD'}

tbl_data = clean_data[fx.keys()].dropna(how='all')
tbl_data.columns = fx.values()
tbl_data.loc[:,'JPY'] *= 100

table = pd.DataFrame()
table[dtxt(tbl_data.index[-1])['day2']] = tbl_data.iloc[-1]
table['1-month moving average'] = tbl_data.iloc[-22:].mean()
table['1-year moving average'] = tbl_data.iloc[-262:].mean()
dec1 = ['JPY', 'KRW']
table.loc[['JPY', 'KRW'],:] = table.loc[['JPY', 'KRW'],:].applymap("{0:.1f}".format)
dec3 = ['GBP', 'EUR', 'CHF', 'AUD', 'NZD', 'CAD', 'SGD']
table.loc[dec3,:] = table.loc[dec3,:].applymap("{0:.3f}".format)
table.loc[~table.index.isin(dec3+dec1)] = table.loc[~table.index.isin(dec3+dec1)].applymap("{0:.2f}".format)
table['1-month percent change'] = (tbl_data.pct_change(22) * 100).iloc[-1].apply("{0:.1f}".format)
table['1-year percent change'] = (tbl_data.pct_change(262) * 100).iloc[-1].apply("{0:.1f}".format)
table['5-year percent change'] = (tbl_data.pct_change(262*5) * 100).iloc[-1].apply("{0:.1f}".format)

table.index = [f'\includegraphics[width=.03\\textwidth]{{data/flags/{cc}}} \ {cc}' for cc in table.index]

(table.to_csv(data_dir / 'fx_table.tex', sep='&', 
              line_terminator='\\\ ', quotechar=' '))

### Jobless claims

In [13]:
data = fred_df('ICNSA', start='2017') / 1000 

data['1M'] = data['VALUE'].rolling(4).mean()

data.div(1000).to_csv(data_dir / 'icsa.csv', index_label='date', float_format='%g')

totval = data['VALUE'].iloc[-1]*1000
datelt = dtxt(data.index[-1])['day1']
latest1m = data["1M"].iloc[-1]*1000
prev1m = data["1M"].iloc[-53]*1000

chval = totval - data['VALUE'].iloc[-2]*1000
chtxt = f'{round(abs(chval),-2):,.0f}'
if chval > 1000:
    change = f'a one-week increase of {chtxt}'
elif chval < -1000:
    change = f'a one-week decrease of {chtxt}'
else:
    change = 'virtually unchanged from the previous week'

text = ('The Department of Labor \href{{https://www.dol.gov/ui/data.pdf}}{{report}} '+
        f'{totval:,.0f} actual new claims for unemployment '+
        'insurance (UI) under state programs (see {\color{cyan!80!blue}\\textbf{---}}) '+
        f'during the week ending {datelt}, {change}. Over the past four weeks, '+
        f'new claims have averaged {round(latest1m,-2):,.0f} per week. During the same '+
        f'four-week period last year, there were an average of {round(prev1m,-2):,.0f} '+
        'new claims per week.')

write_txt(text_dir / 'icsa.txt', text)
print(text)

data = fred_df('CCNSA', start='2017') / 1000 
data.div(1000).to_csv(data_dir / 'ccsa.csv', index_label='date', float_format='%g')


totval = data['VALUE'].iloc[-1]*1000
prval = data['VALUE'].iloc[-2]*1000
datelt = dtxt(data.index[-1])['day1']
prevyrval = data['VALUE'].iloc[-53]*1000
prmoval = data['VALUE'].iloc[-4]*1000
datepr = dtxt(data.index[-53])['day1']

chval = totval - prval
chtxt = f'{round(abs(chval),-2):,.0f}'
if chval > 1000:
    change = f'a one-week increase of {chtxt}'
elif chval < -1000:
    change = f'a one-week decrease of {chtxt}'
else:
    change = 'virtually unchanged from the previous week'

text = (f'For the week ending {datelt}, the Department of Labor '+
        '\href{{https://www.dol.gov/ui/data.pdf}}{{reports}} '+
        f'{totval:,.0f} continued claims for unemployment '+
        'insurance (insured unemployed) under state programs '+
        '(see {\color{green!90!blue}\\textbf{---}})'+
        f', {change}. One year prior, during the week of {datepr}, '+
        f'there were an average of {round(prevyrval,-2):,.0f} '+
        'insured unemployed.')

write_txt(text_dir / 'ccsa.txt', text)
print(text)

The Department of Labor \href{{https://www.dol.gov/ui/data.pdf}}{{report}} 713,824 actual new claims for unemployment insurance (UI) under state programs (see {\color{cyan!80!blue}\textbf{---}}) during the week ending November 28, 2020, a one-week decrease of 122,500. Over the past four weeks, new claims have averaged 756,200 per week. During the same four-week period last year, there were an average of 234,000 new claims per week.
For the week ending November 21, 2020, the Department of Labor \href{{https://www.dol.gov/ui/data.pdf}}{{reports}} 5,240,575 continued claims for unemployment insurance (insured unemployed) under state programs (see {\color{green!90!blue}\textbf{---}}), a one-week decrease of 690,200. One year prior, during the week of November 23, 2019, there were an average of 1,508,300 insured unemployed.


In [14]:
file = 'https://oui.doleta.gov/unemploy/docs/weekly_pandemic_claims.xlsx'
fed_raw = pd.read_excel(file).iloc[1:]
pua_ic = fed_raw.groupby('Rptdate')['PUA IC'].sum()
pua_ic.index = pd.to_datetime(pua_ic.index)
pua_ic.index.name = 'date'
pua_ic.name = 'pua_ic'
cc_raw = fed_raw.dropna(subset=['PUA CC'])
pua_cc = cc_raw.groupby('Rptdate')['PUA CC'].sum()
peuc_cc = cc_raw.groupby('Rptdate')['PEUC CC'].sum()
fed_cc = pua_cc + peuc_cc
fed_cc.index = pd.to_datetime(fed_cc.index)
fed_cc.index.name = 'date'
fed_cc.name = 'fed_cc'

d1 = pd.DataFrame([pua_ic, fed_cc]).T / 1000
d1.div(1000).to_csv(data_dir / 'fed_uic.csv', index_label='date', float_format='%g')

ltdate = dtxt(d1.index[-1])['day1']
prdatecc = dtxt(d1.dropna().index[-4])['day1']
ltdatecc = dtxt(d1.dropna().index[-1])['day1']
pua_ic_lt = f"{round(d1['pua_ic'].iloc[-1] * 1000, -2):,.0f}"
pua_ic_pr = f"{round(d1['pua_ic'].iloc[-2] * 1000, -2):,.0f}"
pua_ic_1m = f"{round(d1['pua_ic'].iloc[-4:].mean() * 1000, -2):,.0f}"
fed_cc_lt = d1.dropna()['fed_cc'].iloc[-1] * 1_000
fed_cc_mo = d1.dropna()['fed_cc'].iloc[-4] * 1_000
fed_cc_ltt = f"{fed_cc_lt:,.0f}"
tot_cc = fed_cc_lt + totval
tot_lt = f"{tot_cc / 1_000_000:,.1f} million"
tot_cc_pr = fed_cc_mo + prmoval
tot_pr = f"{tot_cc_pr / 1_000_000:,.1f} million"

text = (f'Over the week ending {ltdate}, there were {pua_ic_lt} '+
        'initial UI \href{https://oui.doleta.gov/unemploy/DataDashboard.asp}{claims} '+
        'under the Pandemic Unemployment Assistance '+
        '(PUA) program (see {\color{blue!50!purple!80!black}\\textbf{---}}), '+
        f'compared to {pua_ic_pr} during the prior week, '+
        f'and an average of {pua_ic_1m} initial claims per week over the '+
        'past four weeks. \n\n'+
        f'Federal program continuing claims total {fed_cc_ltt} '+
        f'in {ltdatecc} '+
        '(see {\color{green!50!blue}\\textbf{---}}). These include both ' +
        'claims under the PUA program and claims under the Pandemic Emergency '+
        'Unemployment Compensation (PEUC) program. Combining federal program '+
        f'claims with state program claims indicates there are a total '+
        f'of {tot_lt} insured unemployed persons during the week ending {ltdatecc}, '+
        f'compared to {tot_pr} one month prior, during the week ending {prdatecc}. ')

write_txt(text_dir / 'fed_uic.txt', text)

print(text)

Over the week ending November 28, 2020, there were 288,200 initial UI \href{https://oui.doleta.gov/unemploy/DataDashboard.asp}{claims} under the Pandemic Unemployment Assistance (PUA) program (see {\color{blue!50!purple!80!black}\textbf{---}}), compared to 318,900 during the prior week, and an average of 305,900 initial claims per week over the past four weeks. 

Federal program continuing claims total 13,438,518 in November 21, 2020 (see {\color{green!50!blue}\textbf{---}}). These include both claims under the PUA program and claims under the Pandemic Emergency Unemployment Compensation (PEUC) program. Combining federal program claims with state program claims indicates there are a total of 18.7 million insured unemployed persons during the week ending November 21, 2020, compared to 20.1 million one month prior, during the week ending October 31, 2020. 


### VIX (SP500 volatility)

In [20]:
prev = pd.read_excel(data_dir/ 'vixarchive.xls', skiprows=1, index_col='Date')
VIX = 'http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixcurrent.csv'
curr = pd.read_csv(VIX, skiprows=1, index_col='Date', parse_dates=True)
df = (prev.append(curr)['VIX Close'].resample('MS').mean()
      .append(curr['VIX Close'].iloc[-1:]).rename('value'))
df.to_csv(data_dir / 'vix.csv', index_label='date', header='True')

node = end_node(df, 'magenta', percent=False)
write_txt(text_dir / 'vix_node.txt', node)

ldate = dtxt(df.index[-1])['day1']
vallt = df.iloc[-1]
val3y = df.iloc[-37:].mean()

compare = compare_text(vallt, val3y, [3, 15, 30])

text = ('This volatility measure, the VIX index (see {\color{magenta}\\textbf{---}}), '+
        f'was {vallt:.1f} on {ldate}, '+
        f'{compare} the average index value of {val3y:.1f} over the '+
        'past three years.')
write_txt(text_dir / 'vixbasic.txt', text)
print(text)

HTTPError: HTTP Error 404: NOT FOUND

### Oil prices (WTI)

In [17]:
df = fred_df('DCOILWTICO')

latest = df.iloc[-1]

ma = df.resample('MS').mean()

p = ma.append(latest)['VALUE']

(p.to_csv(data_dir / 'wti.csv', index_label='date', header=True))

oneyr = p.pct_change(13).iloc[-1] * 100

threeyr = p.pct_change(37).iloc[-1] * 100

oyt = value_text(oneyr, casual=True)
tyt = value_text(threeyr)
    
ltch = p.loc['2008-06-01'] - p.iloc[-1]

ldate = dtxt(p.index[-1])['day1']

text = (f'As of {ldate}, a barrel of west Texas '+
        f'intermediate (WTI) \\textbf{{crude oil}} sells for \${p.iloc[-1]:.2f} '+
        '(see {\color{red!80!purple}\\textbf{---}}). Over the past '+
        f'year, this measure of oil prices {oyt}. Over the past three years, '+
        f'the price {tyt}. Currently, the WTI price is \${ltch:.2f} per barrel '+
        'below its peak price in June 2008.')

write_txt(text_dir / 'wti.txt', text)
print(text)

As of November 30, 2020, a barrel of west Texas intermediate (WTI) \textbf{crude oil} sells for \$45.20 (see {\color{red!80!purple}\textbf{---}}). Over the past year, this measure of oil prices fell 20.8 percent. Over the past three years, the price decreased 20.2 percent. Currently, the WTI price is \$88.68 per barrel below its peak price in June 2008.


### Inflation Expectations

In [18]:
url = 'http://www.sca.isr.umich.edu/files/tbmpx1px5.csv'
df = pd.read_csv(url)
df.index = pd.to_datetime(df['Month'] + ' ' + df['YYYY'].astype('str'))
data = df['PX5_MD'].loc['2015':]

data.to_csv(data_dir / 'infumich.csv', index_label='date', header='VALUE')

node = end_node(data, 'violet')
write_txt(text_dir / 'infumich_node.txt', node)

data1 = fred_df('T5YIE').loc['2015':,'VALUE']
data1.to_csv(data_dir / 'infbreak.csv', index_label='date', header=True)

node = end_node(data1, 'blue!70!black')
write_txt(text_dir / 'infbreak_node.txt', node)

ldate = dtxt(data.index[-1])['mon1']
pdate = dtxt(data.index[-13])['mon1']
p5val = data.iloc[-61]
lval = data.iloc[-1]
pval = data.iloc[-13]

ldatem = dtxt(data1.index[-1])['day1']
lvalm = data1.iloc[-1]
pdatem = dtxt(data1.index[-262])['day1']
pvalm = data1.iloc[-262]
p5valm = data1.iloc[-(262*5)]

inf_act = pd.read_csv(data_dir / 'cpi.csv')['ALL'].iloc[-60:].mean()

text = (f'As of {ldate}, consumers expect an average inflation rate of {lval} '+
        'percent over the next five years, (see {\color{violet!60!magenta}\\textbf{---}}), '+
        f'compared to an expected rate of {pval} percent '+
        f'in {pdate}. Consumers had expected inflation to average {p5val} percent over the past '+
        f'five years, while actual inflation over the period was {inf_act:.1f} percent.')
write_txt(text_dir / 'inf_exp_cons.txt', text)
print(text)

text2 = (f'As of {ldatem}, markets expect an average inflation rate of {lvalm} percent ' + 
         'over the next five years (see {\color{blue!70!black}\\textbf{---}}), '
         +f'compared to an expected rate of {pvalm} percent on '+
         f'{pdatem}. Markets had expected inflation to average {p5valm} percent over the '+
         f'past five years, five years ago.')
write_txt(text_dir / 'inf_exp_mkts.txt', text2)
print(text2)

As of November 2020, consumers expect an average inflation rate of 2.5 percent over the next five years, (see {\color{violet!60!magenta}\textbf{---}}), compared to an expected rate of 2.5 percent in November 2019. Consumers had expected inflation to average 2.6 percent over the past five years, while actual inflation over the period was 1.8 percent.
As of December 7, 2020, markets expect an average inflation rate of 1.84 percent over the next five years (see {\color{blue!70!black}\textbf{---}}), compared to an expected rate of 1.59 percent on December 6, 2019. Markets had expected inflation to average 1.29 percent over the past five years, five years ago.
